In [86]:

import pandas as pd
import numpy as np
import scipy.stats
from sklearn.metrics.pairwise import cosine_similarity

## Alternativa; Sistema de recomendacion de restaurantes

- El modelo presentado en este notebook es un modelo de filtrado colaborativo que busca la similitud entre usuarios para realizar recomendaciones de productos.

- El filtrado colaborativo hace recomendaciones basadas en interacciones entre el usuario y el producto en el pasado. La suposición detrás del algoritmo es que a usuarios similares les gustan productos similares.

- Para este borrador solo se usaron 2000 datos como prueba. La cantidad total de datos se usarán en la plataforma de VertexAI extrayendo datos de Bigquery.

In [87]:
df = pd.read_parquet("review_sistema.parquet")

In [88]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,eR7ieJD12PUzsYrP8fw6rQ,0,2012-11-02 00:30:24,0,PtiOktOk5COHoNjc6K4gcw,5.0,Great lunch spot @ Citrus Park Mall. Had the 1...,0,---2PmXbF47D870stH1jqA
1,f1Q93O5Hf6GaAvJnTLYmrg,2,2016-04-08 15:44:19,0,SVAAvturmChLoaVuxPz1xQ,5.0,Dined here for the first time with friends. On...,2,---2PmXbF47D870stH1jqA
2,hTA0eCoMdAebXzm4jkx-0A,1,2016-08-21 18:28:20,0,oPJZvPTykI8jQfb38m-4_w,5.0,What a find....right across from the Barefoot ...,2,---2PmXbF47D870stH1jqA
3,KP5OncF2jhT7_J1phHPPww,1,2015-06-27 23:38:13,0,LBxTq5kq_EeazNCbEz0x5Q,5.0,What a wonderful dining experience.... From th...,1,---2PmXbF47D870stH1jqA
4,ReVpjIDupK_VMPn7ZxPvOQ,1,2014-10-27 12:04:47,1,pfVxd_Lm8taCTeqxNzEIrQ,5.0,"As we walked in to the restaurant , I felt I w...",3,---2PmXbF47D870stH1jqA


Para este modelo solo se necesitan los negocios de restaurantes, los usuarios que han calificado y las calificaciones de los usuarios

In [89]:
df_ml = df.loc[:, ["stars", "business_id", "user_id"]]

In [90]:
df_ml.head()

,stars,business_id,user_id
0,5.0,eR7ieJD12PUzsYrP8fw6rQ,---2PmXbF47D870stH1jqA
1,5.0,f1Q93O5Hf6GaAvJnTLYmrg,---2PmXbF47D870stH1jqA
2,5.0,hTA0eCoMdAebXzm4jkx-0A,---2PmXbF47D870stH1jqA
3,5.0,KP5OncF2jhT7_J1phHPPww,---2PmXbF47D870stH1jqA
4,5.0,ReVpjIDupK_VMPn7ZxPvOQ,---2PmXbF47D870stH1jqA


Cantidad de Restaurantes

In [91]:
df_ml["business_id"].nunique()

1807

Cantidad de Usuarios

In [92]:
df_ml["user_id"].nunique()

119

Se crea la siguiente función para asignar a cada id un valor númerico con el propósito de una mejor visualizacion de los datos en las proximos códigos. 

In [93]:
def asignar_clave_numerica(columna):
    claves = {}
    clave_actual = 1
    resultado = []

    for fila in columna:
        if fila not in claves:
            claves[fila] = clave_actual
            clave_actual += 1
        resultado.append(claves[fila])

    return resultado
df_ml["business_id"] = asignar_clave_numerica(df_ml["business_id"])
df_ml["user_id"] = asignar_clave_numerica(df_ml["user_id"])

In [94]:
df_ml["user_id"].nunique()

119

Se crea una Matriz donde las columnas son los restaurantes y las filas son los usuarios, el contenido de cada fila será la calificación que el usuario le ha dando al restaurante. En caso de no existir calificación se mostrará NaN. 

- En este tipo de modelos es normal que los datos útiles sean escasos, sin embargo es posible predecir la calificación de un usuario a traves de tecnicas como; factorización de matriz o correlación de pearson.

In [95]:
matrix = df_ml.pivot_table(index='user_id', columns='business_id', values='stars')
matrix.head()

business_id,1,2,3,4,5,6,7,8,9,10,...,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


El siguiente paso es medir la similitud de los usuarios. En este caso se usa la correlación de pearson

In [96]:
user_similarity = matrix.T.corr()
user_similarity.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,110,111,112,113,114,115,116,117,118,119
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En este paso se usa el ID de usuario 54 como ejemplo para ilustrar cómo encontrar usuarios similares.

Primero debemos excluir el ID de usuario 54 de la lista de usuarios similares y decidir la cantidad de usuarios similares.

In [97]:
# Seleccionar usuario
userid = 54

# Quitar el usuario de la lista
user_similarity.drop(index=userid, inplace=True)

# mostrar similitudes
user_similarity.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,110,111,112,113,114,115,116,117,118,119
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


En la matriz de similitud del usuario, los valores varían de -1 a 1, donde -1 significa similitud opuesta y 1 significa similitud igual.

n = 10 significa los 10 usuarios más similares para el ID de usuario 54.

El filtrado colaborativo basado en usuarios hace recomendaciones basadas en usuarios con gustos similares, por lo que debemos establecer un umbral positivo. Aquí configuramos user_similarity_threshold en 0,3, lo que significa que un usuario debe tener un coeficiente de correlación de Pearson de al menos 0,3 para ser considerado un usuario similar.

Después de establecer el número de usuarios similares y el umbral de similitud, clasificamos el valor de similitud del usuario del más alto al más bajo, luego imprimimos la identificación de los usuarios más similares y el valor de correlación de Pearson.

In [98]:
# Numero de usuarios similares
n = 10

# Umbral
user_similarity_threshold = 0.3

# Obtener el top n de usuarios similares
similar_users = user_similarity[user_similarity[userid]>user_similarity_threshold][userid].sort_values(ascending=False)[:n]

# Ver Usuarios similares
print(f'Los usuarios similares para el usuario {userid} son ', similar_users)

Los usuarios similares para el usuario 54 son  user_id
90     1.000000
29     0.870388
87     0.866025
102    0.500000
Name: 54, dtype: float64


El siguiente paso es reducir el grupo de elementos para simular una recomendación, se hace lo siguiente:

- Eliminar los negocios que haya visitado el usuario seleccionado (ID de usuario 54 en este ejemplo).
- Conservar sólo las calificaciones de usuarios similares

Para realizar lo puntos mencionados se mantiene en la fila solo al usuario seleccionado y se elimina los valores faltantes

In [99]:
picked_userid = matrix[matrix.index == userid].dropna(axis=1, how='all')
picked_userid

business_id,137,285,286,288,289,475,521,540,541,542,...,548,549,550,551,552,553,554,555,556,557
user_id,,,,,,,,,,,,,,,,,,,,,
54,5.0,3.0,4.0,4.0,4.0,4.0,5.0,5.0,4.0,4.0,...,4.0,3.0,3.0,5.0,4.0,5.0,4.0,3.0,4.0,3.0


In [100]:
# business_id que los usuarios similares han calificado 
similar_user_business = matrix[matrix.index.isin(similar_users.index)].dropna(axis=1, how='all')
similar_user_business

business_id,110,116,129,285,286,287,288,289,376,475,...,1466,1467,1468,1469,1525,1526,1527,1528,1643,1644
user_id,,,,,,,,,,,,,,,,,,,,,
29,NaN,NaN,NaN,3.0,5.0,4.0,4.0,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,4.0,2.0,3.0,NaN,3.0,NaN,NaN,4.0,2.0,NaN,...,2.0,5.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,4.5,2.0,2.0,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0


In [101]:
# Elimina los negocios que el usuario seleccionado ha visitado
similar_user_business.drop(picked_userid.columns,axis=1, inplace=True, errors='ignore')

# Tdata
similar_user_business

business_id,110,116,129,287,376,522,523,594,597,603,...,1466,1467,1468,1469,1525,1526,1527,1528,1643,1644
user_id,,,,,,,,,,,,,,,,,,,,,
29,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,4.0,2.0,3.0,NaN,2.0,4.0,NaN,4.0,4.0,4.0,...,2.0,5.0,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,4.5,2.0,2.0,NaN,NaN
102,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,5.0


En este paso se obtiene el top 10 de recomendaciones que se realizan al usuario seleccionado; 54.

 Los elementos recomendados están determinados por el promedio ponderado de la puntuación de similitud del usuario y la calificación del Restauran. Las calificaciones del restauran se ponderan según las puntuaciones de similitud, por lo que los usuarios con mayor similitud obtienen ponderaciones más altas.

Este código recorre los elementos y los usuarios para obtener la puntuación del elemento, clasificar la puntuación de mayor a menor y elegir las 10 mejores restaurantes para recomendar al usuario 54.

In [102]:
# Se declara un diccionario para el puntaje
item_score = {}

# recorrer Restaurantes
for i in similar_user_business.columns:
  # calificaciones de los Restaurantes
  business = similar_user_business[i]
  #  variable que almacena los puntajes
  total = 0
  # variable que almacena el numero de puntajes
  count = 0
  # recorrer usuarios similares
  for u in similar_users.index:
    #Si el restauran tiene calificación
    if pd.isna(business[u]) == False:
      # La puntuación es la suma de la puntuación de similitud del usuario multiplicada por la calificación del restaurante
      score = similar_users[u] * business[u]
      # suma puntuacion a la puntuacion total
      total += score
      # Agrega uno al conteo
      count +=1
  # obtiene la puntuacion media del item
  item_score[i] = total / count

# Convertir diccionario en un dataframe
item_score = pd.DataFrame(item_score.items(), columns=['business', 'score'])
    
# Ordenar
ranked_item_score = item_score.sort_values(by='score', ascending=False)

# Seleccionar top 10
m = 10
ranked_item_score.head(m)

,business,score
484,1525,5.000000
485,1526,4.500000
91,1077,4.330127
252,1238,4.330127
431,1417,4.330127
126,1112,4.330127
122,1108,4.330127
231,1217,4.330127
437,1423,4.330127
238,1224,4.330127
